In [18]:
import os
import re
import sys
import csv
import time
import tqdm
import nltk
import math
import string
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn import metrics
from sklearn import tree
from sklearn import datasets
from sklearn.svm import LinearSVC
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import Normalizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import BernoulliNB
from sklearn.decomposition import TruncatedSVD
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import KFold
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords
from nltk.stem import *
from nltk.corpus import wordnet
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import normalize
from sklearn.feature_selection import chi2
from scipy.stats import uniform
from scipy.stats import randint
from spellchecker import SpellChecker #need to install for some computers
from Test import * 


feature_names = []

#vectorizer = TfidfVectorizer(ngram_range=(1,1),stop_words='english',norm='l2',sublinear_tf=False)
vectorizer = CountVectorizer(ngram_range=(1,1),stop_words='english')

corpus_train = pd.read_csv("reddit_train.csv",usecols=['comments','subreddits'],delimiter=',')
corpus_test = pd.read_csv("reddit_test.csv",usecols=['comments'],delimiter=',')
english_words = set(nltk.corpus.words.words()\
                    + nltk.corpus.gutenberg.words()\
                    + nltk.corpus.webtext.words()\
                    + nltk.corpus.nps_chat.words()\
                    + nltk.corpus.brown.words())
#                     + nltk.corpus.reuters.words()\

#spell = SpellChecker()

# def is_english_word(x):
#     return (spell.correction(x) in english_words)

# important for cleaning non-english tokens
#spell = SpellChecker()
# delete_list = ["aaa" ,"bbb","ccc","ddd","eee","fff","ggg","hhh","iii","jjj","kkk","lll","mmm","nnn","ooo","ppp","qqq","rrr","sss","ttt"\
#                ,"uuu","vvv","www","xxx","yyy","zzz"]

def misspell(x):
    if "aaa" not in x and "bbb" not in x and "ccc" not in x and "ddd" not in x \
    and "eee" not in x and "fff" not in x and "ggg" not in x and "hhh" not in x \
    and "iii" not in x and "jjj" not in x and "kkk" not in x and "lll" not in x and "mmm" not in x and "nnn" not in x \
    and "ooo" not in x and "ppp" not in x and "qqq" not in x and "rrr" not in x and "sss" not in x and "ttt" not in x \
    and "uuu" not in x and "vvv" not in x and "www" not in x and "xxx" not in x and "yyy" not in x and "zzz" not in x \
    and "aa" not in x and "zz" not in x:
        return True
    else:
        return False
    

# delete_list = ["aaa" not in x,"bbb" not in x,"ccc" not in x,"ddd" not in x,"eee" not in x,"fff" not in x,"ggg" not in x,\
#                "hhh" not in x,"iii" not in x,"jjj" not in x,"kkk" not in x,"lll" not in x,"mmm" not in x,"nnn" not in x,\
#                "ooo" not in x,"ppp" not in x,"qqq" not in x,"rrr" not in x,"sss" not in x,"ttt" not in x
#                ,"uuu" not in x,"vvv" not in x,"www" not in x,"xxx" not in x,"yyy" not in x,"zzz" not in x]

# a helper function to process one comment
def preprocess_text(text): 
    text = text.lower().split()
    stops = set(stopwords.words("english"))
#     text = [w for w in text if not w in stops and len(w) >= 3]
    text = " ".join(text)
    text = re.sub(r"what's", "what is ", text)
    text = re.sub(r"\'s", " ", text)
    text = re.sub(r"\'ve", " have ", text)
    text = re.sub(r"n't", " not ", text)
    text = re.sub(r"i'm", "i am ", text)
    text = re.sub(r"\'re", " are ", text)
    text = re.sub(r"\'d", " would ", text)
    text = re.sub(r"\'ll", " will ", text)
    text = re.sub(r",", " ", text)
    text = re.sub(r"\.", " ", text)
    text = re.sub(r"!", " ! ", text)
    text = re.sub(r"\/", " ", text)
    text = re.sub(r"\^", " ^ ", text)
    text = re.sub(r"\+", " + ", text)
    text = re.sub(r"\-", " - ", text)
    text = re.sub(r"\=", " = ", text)
    text = re.sub(r"'", " ", text)
    text = re.sub(r"^https?:\/\/.*[\r\n]*","", text)
    text = re.sub(r'[^a-zA-Z0-9\s]', ' ',text)
    text = text.split()
    
    # lemmatization
    lemma = nltk.wordnet.WordNetLemmatizer()
    text = [lemma.lemmatize(word) for word in text]
    
#     # stemming
#     text = [PorterStemmer().stem(word) for word in text]
    
    text_final = []
    
    # clean all non-English words, numbers, and other weirdos, stopwords
    for x in text:
        #x = spell.correction(x)
        if x.isalpha() and x not in stops and len(x)<20 and len(x) > 2 and misspell(x):# and is_english_word(x):
            text_final.append(x)
    
    text = " ".join(text_final)
    return text


# the major function to process the training dataset
# returns (1) a matrix of all training features x (2) a numpy array of y labels
def preprocess():
    df = corpus_train.copy()
    df['comments'] = df['comments'].map(lambda x: preprocess_text(x))
    y_train = df["subreddits"].to_numpy()
    x_train = vectorizer.fit_transform(df['comments'])
    
    print(vectorizer.get_feature_names())
    global feature_names
    feature_names = vectorizer.get_feature_names()
#     featname=vectorizer.get_feature_names()
#     chi_squared,pval = chi2(x_train, y_train)
#     featname = pd.DataFrame(featname)
#     chi_2 = pd.DataFrame(chi_squared)
#     pval = pd.DataFrame(pval)

#     data = pd.concat([featname,chi_2,pval],axis=1)
#     data.columns = ["word","chi_squared","pval"]
#     data = data.sort_values("pval",axis=0)

#     to_delete = list(data.index[10000:])     # 保留10000-15000之间最高
#     all_cols = np.arange(x_train.shape[1])
#     cols_to_keep = np.where(np.logical_not(np.in1d(all_cols, to_delete)))[0]
#     x_train = x_train[:, cols_to_keep]
    return x_train, y_train


# function to process the testing set
# returns the matrix of all testing features
def preprocess_testing():
    df = corpus_test.copy()
    df['comments'] = df['comments'].map(lambda x: preprocess_text(x))
    #y_train = df["subreddits"].to_numpy()
    x_train = vectorizer.fit_transform(df['comments'])
    return x_train
    

In [19]:
x,y = preprocess()
print(x.shape)

print(x)



['aba', 'aback', 'abadi', 'abam', 'abandon', 'abandoned', 'abandoning', 'abandonment', 'abandonned', 'abandonware', 'abaya', 'abb', 'abbas', 'abbey', 'abbotsford', 'abbreviated', 'abbreviation', 'abby', 'abc', 'abck', 'abd', 'abdelhak', 'abdelkader', 'abdicate', 'abdomen', 'abdominal', 'abducted', 'abducting', 'abduction', 'abductive', 'abdul', 'abdullah', 'abe', 'abeast', 'abed', 'abel', 'abell', 'abendblatt', 'aberation', 'aberdeen', 'aberg', 'aberration', 'abetting', 'abeyta', 'abgelehnt', 'abgrenzung', 'abhor', 'abhorrent', 'abhors', 'abidal', 'abide', 'abiding', 'abilify', 'abilites', 'abilitie', 'ability', 'abillities', 'abilties', 'abin', 'abingdon', 'abington', 'abit', 'abitur', 'abject', 'ablaze', 'able', 'abled', 'ableton', 'abliged', 'abn', 'abnormal', 'abnormality', 'abnormally', 'aboard', 'abode', 'abogado', 'aboiut', 'abolish', 'abolished', 'abolition', 'abolitionist', 'abolve', 'abominably', 'abomination', 'aboot', 'aboriginal', 'aborigine', 'aborigionals', 'abort', 'abo


(70000, 57430)
  (0, 22973)	1
  (0, 6462)	3
  (0, 10367)	1
  (0, 1888)	1
  (0, 41636)	1
  (0, 37128)	1
  (0, 46893)	1
  (0, 26124)	1
  (0, 30242)	1
  (0, 47897)	1
  (0, 20369)	1
  (0, 55951)	1
  (0, 19417)	1
  (0, 51539)	1
  (0, 50637)	1
  (0, 14997)	1
  (0, 35021)	1
  (0, 51767)	1
  (0, 4649)	1
  (0, 56743)	1
  (0, 49501)	1
  (0, 12684)	1
  (0, 50247)	1
  (0, 9505)	1
  (0, 42071)	1
  :	:
  (69998, 38849)	1
  (69998, 50407)	1
  (69998, 22108)	1
  (69998, 28745)	1
  (69998, 1788)	2
  (69998, 589)	1
  (69998, 29998)	1
  (69998, 1791)	1
  (69998, 36054)	1
  (69998, 46791)	1
  (69998, 32255)	1
  (69998, 22147)	1
  (69998, 49694)	1
  (69999, 29388)	1
  (69999, 17383)	1
  (69999, 7448)	1
  (69999, 21930)	1
  (69999, 30592)	1
  (69999, 7457)	3
  (69999, 50883)	1
  (69999, 21947)	1
  (69999, 22371)	1
  (69999, 36472)	1
  (69999, 28114)	1
  (69999, 43469)	1


In [20]:
clf = MultinomialNB(alpha=0.0565)
#x,y = preprocess()

model_validate(clf,x,y)

# clf = BernoulliNB(alpha=1)
# #x,y = preprocess()

# model_validate(clf,x,y)



Result of k-fold cross validation on  MultinomialNB
0.5404571428571427


In [21]:
from tqdm import tqdm

index_to_delete = []
index_to_combine = []


for i in tqdm(range(len(feature_names)-1)):
    same_root = [i]
    for j in range(i+1,len(feature_names)):
        if(feature_names[i][0]!=feature_names[j][0]):
            break
        else:
            if PorterStemmer().stem(feature_names[i]) == PorterStemmer().stem(feature_names[j]):
                same_root.append(j)

    index_to_combine.append(same_root)




# from spellchecker import SpellChecker
# spell = SpellChecker()

# wrong_word_index_list = []
# for i in tqdm(range(len(feature_names))):
#     if len(feature_names[i])<=10:
#         if (spell.correction(feature_names[i]) not in english_words):
#             wrong_word_index_list.append(i)

# print(wrong_word_index_list)






  0%|                                                                             | 67/57429 [00:09<2:21:53,  6.74it/s]


  0%|▏                                                                           | 135/57429 [00:19<2:15:00,  7.07it/s]


  0%|▎                                                                           | 203/57429 [00:28<2:14:39,  7.08it/s]


  0%|▎                                                                           | 233/57429 [00:32<2:10:12,  7.32it/s]

KeyboardInterrupt: 

In [ ]:
#print(index_to_combine)
#index_to_combine_plus = [line for line in index_to_combine if len(line)>2]
filtered_lines = list(filter(lambda x: len(x) > 1, index_to_combine))
to_delete_index = []
for i in range(len(filtered_lines)-1):
    for j in range(i+1,min(len(filtered_lines),i+10)):
        if set(filtered_lines[j]) <= set(filtered_lines[i]):
            to_delete_index.append(j)
    

In [ ]:
filtered_lines

In [ ]:

len(filtered_lines) - len(set(to_delete_index))

In [ ]:
new_line = []
main_cols = []
for i in range(len(filtered_lines)):
    if i not in set(to_delete_index):
        new_line.append(filtered_lines[i])
        main_cols.append(filtered_lines[i][0])

In [17]:
new_line[-10:]

[[57165, 57166],
 [57183, 57184],
 [57192, 57193],
 [57209, 57210],
 [57217, 57218],
 [57244, 57245],
 [57291, 57294],
 [57322, 57323],
 [57334, 57335, 57339],
 [57348, 57349, 57351]]

In [10]:
new_line #######
#x,y = preprocess()
print(x.shape)

(70000, 10000)


In [22]:
temp = x

In [23]:
print(temp.shape)
print(len(new_line))

(70000, 57430)
6994


In [146]:
from scipy.sparse import csr_matrix
#len(temp[:,2666].toarray())
temp[:,4] = csr_matrix(temp[:,4].toarray()+temp[:,4].toarray())


In [24]:
from scipy.sparse import csr_matrix
for i in tqdm(range(len(new_line))):
    for j in range(1,len(new_line[i])):
        temp[:,new_line[i][0]] = csr_matrix(temp[:,new_line[i][0]].toarray() + temp[:,new_line[i][j]].toarray())



  0%|                                                                                         | 0/6994 [00:00<?, ?it/s]D:\PROGRAMS\Anaconda\lib\site-packages\scipy\sparse\_index.py:118: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_arrayXarray_sparse(i, j, x)


  0%|                                                                                 | 3/6994 [00:00<04:37, 25.20it/s]

  0%|                                                                                | 10/6994 [00:00<03:50, 30.30it/s]

  0%|▏                                                                               | 15/6994 [00:00<03:25, 33.96it/s]

  0%|▏                                                                               | 18/6994 [00:00<03:43, 31.28it/s]

  0%|▎                                                                               | 25/6994 [00:00<03:23, 34.32it/s]

  0%|▎                                          

  4%|███▏                                                                           | 285/6994 [00:09<03:14, 34.47it/s]

  4%|███▎                                                                           | 290/6994 [00:09<02:57, 37.78it/s]

  4%|███▎                                                                           | 295/6994 [00:09<03:26, 32.39it/s]

  4%|███▍                                                                           | 299/6994 [00:09<03:20, 33.35it/s]

  4%|███▍                                                                           | 304/6994 [00:09<03:01, 36.83it/s]

  4%|███▍                                                                           | 308/6994 [00:09<03:17, 33.78it/s]

  4%|███▌                                                                           | 312/6994 [00:09<03:27, 32.15it/s]

  5%|███▌                                                                           | 317/6994 [00:09<03:10, 35.05it/s]

  5%|███▋                       

  8%|██████▋                                                                        | 589/6994 [00:18<02:36, 40.95it/s]

  8%|██████▋                                                                        | 594/6994 [00:18<02:34, 41.31it/s]

  9%|██████▊                                                                        | 599/6994 [00:18<02:29, 42.73it/s]

  9%|██████▊                                                                        | 604/6994 [00:18<02:27, 43.39it/s]

  9%|██████▉                                                                        | 609/6994 [00:18<02:38, 40.23it/s]

  9%|██████▉                                                                        | 614/6994 [00:18<03:09, 33.72it/s]

  9%|██████▉                                                                        | 619/6994 [00:18<02:56, 36.11it/s]

  9%|███████                                                                        | 623/6994 [00:18<03:04, 34.55it/s]

  9%|███████                    

 13%|█████████▉                                                                     | 882/6994 [00:26<03:52, 26.29it/s]

 13%|██████████                                                                     | 887/6994 [00:27<03:56, 25.85it/s]

 13%|██████████                                                                     | 890/6994 [00:27<04:16, 23.83it/s]

 13%|██████████                                                                     | 893/6994 [00:27<04:16, 23.77it/s]

 13%|██████████▏                                                                    | 897/6994 [00:27<03:48, 26.65it/s]

 13%|██████████▏                                                                    | 900/6994 [00:27<04:11, 24.22it/s]

 13%|██████████▏                                                                    | 903/6994 [00:27<04:00, 25.30it/s]

 13%|██████████▏                                                                    | 906/6994 [00:27<04:06, 24.72it/s]

 13%|██████████▎                

 16%|████████████▍                                                                 | 1120/6994 [00:36<09:36, 10.19it/s]

 16%|████████████▌                                                                 | 1124/6994 [00:36<07:27, 13.12it/s]

 16%|████████████▌                                                                 | 1127/6994 [00:37<07:07, 13.71it/s]

 16%|████████████▌                                                                 | 1129/6994 [00:37<09:46, 10.00it/s]

 16%|████████████▌                                                                 | 1131/6994 [00:37<08:51, 11.04it/s]

 16%|████████████▋                                                                 | 1133/6994 [00:37<08:25, 11.60it/s]

 16%|████████████▋                                                                 | 1135/6994 [00:37<09:15, 10.55it/s]

 16%|████████████▋                                                                 | 1137/6994 [00:38<08:54, 10.96it/s]

 16%|████████████▋              

...

 81%|███████████████████████████████████████████████████████████████               | 5650/6994 [04:40<00:51, 26.34it/s]

 81%|███████████████████████████████████████████████████████████████               | 5653/6994 [04:40<00:53, 24.87it/s]

 81%|███████████████████████████████████████████████████████████████               | 5656/6994 [04:40<00:57, 23.38it/s]

 81%|███████████████████████████████████████████████████████████████               | 5660/6994 [04:40<00:53, 24.87it/s]

 81%|███████████████████████████████████████████████████████████████▏              | 5663/6994 [04:40<00:58, 22.56it/s]

 81%|███████████████████████████████████████████████████████████████▏              | 5667/6994 [04:40<00:52, 25.34it/s]

 81%|███████████████████████████████████████████████████████████████▏              | 5670/6994 [04:40<00:58, 22.48it/s]

 81%|███████████████████████████████████████████████████████████████▎              | 5674/6994 [04:41<00:55, 23.78it/s]

 81%|███████████████████████████

 84%|█████████████████████████████████████████████████████████████████▊            | 5901/6994 [04:49<00:37, 28.96it/s]

 84%|█████████████████████████████████████████████████████████████████▊            | 5905/6994 [04:49<00:39, 27.72it/s]

 84%|█████████████████████████████████████████████████████████████████▉            | 5908/6994 [04:49<00:39, 27.81it/s]

 85%|█████████████████████████████████████████████████████████████████▉            | 5911/6994 [04:49<00:39, 27.60it/s]

 85%|█████████████████████████████████████████████████████████████████▉            | 5915/6994 [04:49<00:38, 27.80it/s]

 85%|██████████████████████████████████████████████████████████████████            | 5920/6994 [04:49<00:35, 30.16it/s]

 85%|██████████████████████████████████████████████████████████████████            | 5924/6994 [04:49<00:33, 31.82it/s]

 85%|██████████████████████████████████████████████████████████████████            | 5928/6994 [04:50<00:37, 28.68it/s]

 85%|███████████████████████████

 88%|████████████████████████████████████████████████████████████████████▍         | 6138/6994 [04:58<00:27, 31.38it/s]

 88%|████████████████████████████████████████████████████████████████████▌         | 6143/6994 [04:58<00:24, 34.06it/s]

 88%|████████████████████████████████████████████████████████████████████▌         | 6147/6994 [04:58<00:32, 26.11it/s]

 88%|████████████████████████████████████████████████████████████████████▌         | 6151/6994 [04:58<00:32, 25.96it/s]

 88%|████████████████████████████████████████████████████████████████████▋         | 6155/6994 [04:59<00:30, 27.80it/s]

 88%|████████████████████████████████████████████████████████████████████▋         | 6160/6994 [04:59<00:26, 31.11it/s]

 88%|████████████████████████████████████████████████████████████████████▋         | 6164/6994 [04:59<00:29, 28.60it/s]

 88%|████████████████████████████████████████████████████████████████████▊         | 6168/6994 [04:59<00:27, 29.97it/s]

 88%|███████████████████████████

 92%|███████████████████████████████████████████████████████████████████████▋      | 6426/6994 [05:07<00:17, 31.72it/s]

 92%|███████████████████████████████████████████████████████████████████████▋      | 6430/6994 [05:07<00:17, 33.10it/s]

 92%|███████████████████████████████████████████████████████████████████████▊      | 6435/6994 [05:08<00:15, 36.22it/s]

 92%|███████████████████████████████████████████████████████████████████████▊      | 6439/6994 [05:08<00:16, 33.84it/s]

 92%|███████████████████████████████████████████████████████████████████████▉      | 6445/6994 [05:08<00:14, 38.84it/s]

 92%|███████████████████████████████████████████████████████████████████████▉      | 6450/6994 [05:08<00:13, 40.85it/s]

 92%|███████████████████████████████████████████████████████████████████████▉      | 6455/6994 [05:08<00:14, 36.29it/s]

 92%|████████████████████████████████████████████████████████████████████████      | 6460/6994 [05:08<00:13, 39.07it/s]

 92%|███████████████████████████

 97%|███████████████████████████████████████████████████████████████████████████▍  | 6762/6994 [05:16<00:06, 36.74it/s]

 97%|███████████████████████████████████████████████████████████████████████████▍  | 6767/6994 [05:16<00:05, 38.88it/s]

 97%|███████████████████████████████████████████████████████████████████████████▌  | 6772/6994 [05:16<00:05, 39.72it/s]

 97%|███████████████████████████████████████████████████████████████████████████▌  | 6778/6994 [05:16<00:04, 44.19it/s]

 97%|███████████████████████████████████████████████████████████████████████████▋  | 6784/6994 [05:17<00:04, 46.04it/s]

 97%|███████████████████████████████████████████████████████████████████████████▋  | 6789/6994 [05:17<00:04, 43.99it/s]

 97%|███████████████████████████████████████████████████████████████████████████▊  | 6794/6994 [05:17<00:04, 41.97it/s]

 97%|███████████████████████████████████████████████████████████████████████████▊  | 6799/6994 [05:17<00:04, 40.80it/s]

 97%|███████████████████████████

In [25]:
to_delete_from_matrix=[]
for i in range(len(new_line)):
    for j in range(1,len(new_line[i])):
        to_delete_from_matrix.append(new_line[i][j])
to_delete_from_matrix= list(set(to_delete_from_matrix))

In [26]:
to_delete = to_delete_from_matrix
all_cols = np.arange(temp.shape[1])
cols_to_keep = np.where(np.logical_not(np.in1d(all_cols, to_delete)))[0]
temp = temp[:, cols_to_keep]

In [39]:
feature_names_new = np.array(feature_names)[cols_to_keep]

In [40]:
feature_names_new.shape

(45018,)

In [27]:
temp.shape

(70000, 45018)

In [32]:
clf = MultinomialNB(alpha=0.0565)
#x,y = preprocess()

model_validate(clf,temp,y)

Result of k-fold cross validation on  MultinomialNB
0.5442714285714285


In [42]:
featname=feature_names_new
chi_squared,pval = chi2(temp, y)
featname = pd.DataFrame(featname)
chi_2 = pd.DataFrame(chi_squared)
pval = pd.DataFrame(pval)

data = pd.concat([featname,chi_2,pval],axis=1)
data.columns = ["word","chi_squared","pval"]
data = data.sort_values("pval",axis=0)
data

,word,chi_squared,pval
26186,nba,3478.876923,0.000000
2027,askreddit,3149.395349,0.000000
7518,compose,3211.485876,0.000000
36088,smoke,8009.905882,0.000000
23057,lyric,1726.038760,0.000000
...,...,...,...
32225,reimburse,10.000000,0.952946
94,absent,10.000000,0.952946
27389,oddly,9.384615,0.966531
120,abundance,9.000000,0.973479


In [66]:
to_delete = list(data.index[10000:])     # 保留10000-15000之间最高
all_cols = np.arange(temp.shape[1])
cols_to_keep = np.where(np.logical_not(np.in1d(all_cols, to_delete)))[0]
temp2 = temp[:, cols_to_keep]

In [72]:
from sklearn.naive_bayes import GaussianNB
clf = MultinomialNB()
#x,y = preprocess()

model_validate(clf,temp2,y)

Result of k-fold cross validation on  MultinomialNB
0.5428000000000001


In [73]:
x,y = preprocess()
clf = MultinomialNB()
#x,y = preprocess()

model_validate(clf,x,y)

['aba', 'aback', 'abadi', 'abam', 'abandon', 'abandoned', 'abandoning', 'abandonment', 'abandonned', 'abandonware', 'abaya', 'abb', 'abbas', 'abbey', 'abbotsford', 'abbreviated', 'abbreviation', 'abby', 'abc', 'abck', 'abd', 'abdelhak', 'abdelkader', 'abdicate', 'abdomen', 'abdominal', 'abducted', 'abducting', 'abduction', 'abductive', 'abdul', 'abdullah', 'abe', 'abeast', 'abed', 'abel', 'abell', 'abendblatt', 'aberation', 'aberdeen', 'aberg', 'aberration', 'abetting', 'abeyta', 'abgelehnt', 'abgrenzung', 'abhor', 'abhorrent', 'abhors', 'abidal', 'abide', 'abiding', 'abilify', 'abilites', 'abilitie', 'ability', 'abillities', 'abilties', 'abin', 'abingdon', 'abington', 'abit', 'abitur', 'abject', 'ablaze', 'able', 'abled', 'ableton', 'abliged', 'abn', 'abnormal', 'abnormality', 'abnormally', 'aboard', 'abode', 'abogado', 'aboiut', 'abolish', 'abolished', 'abolition', 'abolitionist', 'abolve', 'abominably', 'abomination', 'aboot', 'aboriginal', 'aborigine', 'aborigionals', 'abort', 'abo


Result of k-fold cross validation on  MultinomialNB
0.5399571428571428


In [142]:
for i in tqdm(range(len(new_line))):
    for j in range(1,len(new_line[i])):
        temp[:,new_line[i][0]] = csr_matrix(temp[:,new_line[i][j]].toarray() + temp[:,new_line[i][0]].toarray())





  0%|                                                                                         | 0/6994 [00:00<?, ?it/s]

ValueError: indices and data should have the same size

In [117]:
temp[:,0].toarray()

array([[0.],
       [0.],
       [0.],
       ...,
       [0.],
       [0.],
       [0.]])

In [98]:
set([1,2]) <= set([1,2,3])

True

In [67]:
#print(index_to_combine)
x,y = preprocess()

print(x)




['aba', 'aback', 'abadi', 'abam', 'abandon', 'abandoned', 'abandoning', 'abandonment', 'abandonned', 'abandonware', 'abaya', 'abb', 'abbas', 'abbey', 'abbotsford', 'abbreviated', 'abbreviation', 'abby', 'abc', 'abck', 'abd', 'abdelhak', 'abdelkader', 'abdicate', 'abdomen', 'abdominal', 'abducted', 'abducting', 'abduction', 'abductive', 'abdul', 'abdullah', 'abe', 'abeast', 'abed', 'abel', 'abell', 'abendblatt', 'aberation', 'aberdeen', 'aberg', 'aberration', 'abetting', 'abeyta', 'abgelehnt', 'abgrenzung', 'abhor', 'abhorrent', 'abhors', 'abidal', 'abide', 'abiding', 'abilify', 'abilites', 'abilitie', 'ability', 'abillities', 'abilties', 'abin', 'abingdon', 'abington', 'abit', 'abitur', 'abject', 'ablaze', 'able', 'abled', 'ableton', 'abliged', 'abn', 'abnormal', 'abnormality', 'abnormally', 'aboard', 'abode', 'abogado', 'aboiut', 'abolish', 'abolished', 'abolition', 'abolitionist', 'abolve', 'abominably', 'abomination', 'aboot', 'aboriginal', 'aborigine', 'aborigionals', 'abort', 'abo


  (0, 29360)	0.20022594191910897
  (0, 14190)	0.15775115563721837
  (0, 7907)	0.11740190288167779
  (0, 42071)	0.1723313388019609
  (0, 9505)	0.15701205849058875
  (0, 50247)	0.13392036335509133
  (0, 12684)	0.19056621306873878
  (0, 49501)	0.23458446228031543
  (0, 56743)	0.08516746372577937
  (0, 4649)	0.09539836510311284
  (0, 51767)	0.22717098530336569
  (0, 35021)	0.15775115563721837
  (0, 14997)	0.1977067724450827
  (0, 50637)	0.07728236382946274
  (0, 51539)	0.16573477387875873
  (0, 19417)	0.08938147609101936
  (0, 55951)	0.11864793162774175
  (0, 20369)	0.1760204656970901
  (0, 47897)	0.14140138741898675
  (0, 30242)	0.2412815872074171
  (0, 26124)	0.18691498233158588
  (0, 46893)	0.16558409629962711
  (0, 37128)	0.07666144854857686
  (0, 41636)	0.12030554299058105
  (0, 1888)	0.13776730808959703
  :	:
  (69998, 28745)	0.17320360534707682
  (69998, 22108)	0.19524994188505868
  (69998, 50407)	0.15127222268163704
  (69998, 38849)	0.18427540349679816
  (69998, 33754)	0.165390761

MemoryError: 

In [36]:
all_cols = np.arange(x.shape[1])
cols_to_keep = np.where(np.logical_not(np.in1d(all_cols, wrong_word_index_list)))[0]
x = x[:, cols_to_keep]

In [37]:
print(x.shape)

(70000, 40956)


In [38]:
clf = MultinomialNB(alpha=0.0565)
#x,y = preprocess()

model_validate(clf,x,y)

Result of k-fold cross validation on  MultinomialNB
0.49660000000000004


In [33]:
x,y = preprocess()
featname=vectorizer.get_feature_names()
#print(len(featname))
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import normalize
from sklearn.feature_selection import chi2

chi2,pval = chi2(x, y)
featname = pd.DataFrame(featname)
chi2 = pd.DataFrame(chi2)
pval = pd.DataFrame(pval)

data=pd.concat([featname,chi2,pval],axis=1)
data.columns = ["word","chi2","pval"]
data=data.sort_values("pval",axis=0)

to_delete = list(data.index[10000:])     # 保留10000-15000之间最高

all_cols = np.arange(x.shape[1])
cols_to_keep = np.where(np.logical_not(np.in1d(all_cols, to_delete)))[0]
m = x[:, cols_to_keep]

clf = MultinomialNB(alpha=0.15,fit_prior=True)

model_validate(clf,m,y)



KeyboardInterrupt: 

In [119]:
type(x)

scipy.sparse.csr.csr_matrix

In [120]:
print(x.shape)

(70000, 58846)


In [121]:
# import scipy
# def delete_rows_csr(mat, indices):
#     """
#     Remove the rows denoted by ``indices`` form the CSR sparse matrix ``mat``.
#     """
#     if not isinstance(mat, scipy.sparse.csr_matrix):
#         raise ValueError("works only for CSR format -- use .tocsr() first")
#     #indices = list(indices)
#     mask = np.ones(mat.shape, dtype=bool)
#     mask[:,indices] = False
#     return mat[mask]

to_delete = list(data.index[10000:])     # 保留10000-15000之间最高

all_cols = np.arange(x.shape[1])
cols_to_keep = np.where(np.logical_not(np.in1d(all_cols, to_delete)))[0]
m = x[:, cols_to_keep]

In [122]:
#print(data['pval'])

#print(sorted(x,reverse=True))
print(m.shape)



#clf = LogisticRegression()
from sklearn.svm import SVC
#clf = SVC(gamma='auto')
clf = MultinomialNB(alpha=0.15,fit_prior=False)

(70000, 10000)


In [123]:
model_validate(clf,m,y)

Result of k-fold cross validation on  MultinomialNB
0.5860285714285715


In [106]:
print(new_x.shape)

NameError: name 'new_x' is not defined

In [73]:
# print(type(x))
# print(x)
matrix = np.asmatrix(x)
print(matrix)

[[<70000x59109 sparse matrix of type '<class 'numpy.float64'>'
	with 1169985 stored elements in Compressed Sparse Row format>]]


In [ ]:
x,y = preprocess()
print(x.shape)
y2 = preprocess_testing()
print(y2.shape)

In [175]:
print(vectorizer.get_feature_names())

  (0, 22632)	1
  (0, 6394)	3
  (0, 25040)	1
  (0, 49543)	5
  (0, 10205)	1
  (0, 1908)	1
  (0, 40958)	1
  (0, 36519)	1
  (0, 25734)	1
  (0, 49523)	1
  (0, 29720)	1
  (0, 18126)	3
  (0, 47055)	1
  (0, 20038)	1
  (0, 53925)	3
  (0, 19121)	1
  (0, 19628)	1
  (0, 49755)	1
  (0, 14772)	1
  (0, 18629)	1
  (0, 34423)	1
  (0, 50873)	1
  (0, 4620)	1
  (0, 36054)	1
  (0, 34386)	2
  (0, 55815)	1
  (0, 6632)	1
  (0, 48625)	1
  (0, 12488)	1
  (0, 49354)	1
  (0, 9358)	1
  (0, 32228)	1
  (0, 31961)	1
  (0, 41375)	1
  (0, 7811)	1
  (0, 50212)	1
  (0, 13961)	1
  (0, 28870)	1
{'movies', 'baseball', 'nba', 'canada', 'hockey', 'Overwatch', 'GlobalOffensive', 'wow', 'gameofthrones', 'worldnews', 'Music', 'funny', 'nfl', 'europe', 'soccer', 'trees', 'AskReddit', 'anime', 'conspiracy', 'leagueoflegends'}


In [152]:
print(vectorizer.get_feature_names())

['aa', 'aaa', 'aaaa', 'aaaaa', 'aaaaaa', 'aaaaaaaaaaaaand', 'aaaaaaaaahq', 'aaaaaaaabeq', 'aaaaaaaacxi', 'aaaaaaaaezo', 'aaaaaaaafii', 'aaaaaaaamza', 'aaaaaaaannd', 'aaaaaaaaz', 'aaaaaaabheo', 'aaaaaand', 'aaaaand', 'aaaages', 'aaaah', 'aaaand', 'aaaannnd', 'aaah', 'aaahahaha', 'aaand', 'aaannnd', 'aaawhitney', 'aabb', 'aac', 'aacnjournals', 'aaf', 'aafc', 'aah', 'aahhh', 'aahhhh', 'aaliyah', 'aam', 'aan', 'aang', 'aangechouwen', 'aardman', 'aaron', 'aarp', 'aarya', 'aasaults', 'aat', 'aatrox', 'aau', 'aav', 'aawe', 'ab', 'aba', 'aback', 'abadi', 'abam', 'abandon', 'abandoned', 'abandoning', 'abandonment', 'abandonned', 'abandonware', 'abaya', 'abb', 'abbas', 'abbey', 'abbreviated', 'abbreviation', 'abby', 'abc', 'abck', 'abd', 'abdelhak', 'abdelkader', 'abdicate', 'abdomen', 'abdominal', 'abducted', 'abducting', 'abduction', 'abductive', 'abdul', 'abdullah', 'abe', 'abeast', 'abed', 'abel', 'abell', 'abendblatt', 'aberation', 'aberdeen', 'aberg', 'aberration', 'abetting', 'abeyta', 'a

In [11]:
vectorizer = CountVectorizer()

corpus = [ 'This is a sentence',
           'Another sentence is here',
           'Wait for another sentence',
           'The sentence is coming',
           'The sentence has come'
         ]

x = vectorizer.fit_transform(corpus)
print(pd.DataFrame(x.A, columns=vectorizer.get_feature_names()).to_string())

   another  come  coming  for  has  here  is  sentence  the  this  wait
0        0     0       0    0    0     0   1         1    0     1     0
1        1     0       0    0    0     1   1         1    0     0     0
2        1     0       0    1    0     0   0         1    0     0     1
3        0     0       1    0    0     0   1         1    1     0     0
4        0     1       0    0    1     0   0         1    1     0     0


In [13]:
print(x.A)

[[0 0 0 0 0 0 1 1 0 1 0]
 [1 0 0 0 0 1 1 1 0 0 0]
 [1 0 0 1 0 0 0 1 0 0 1]
 [0 0 1 0 0 0 1 1 1 0 0]
 [0 1 0 0 1 0 0 1 1 0 0]]


In [19]:
from spellchecker import SpellChecker
spell = SpellChecker()
x = spell.correction("admires")
print(x)

admires


In [53]:
PorterStemmer().stem("abandonware")
#PorterStemmer().stem("administration")
# lemma = nltk.wordnet.WordNetLemmatizer()
# lemma.lemmatize("administr")

# x = spell.correction("administr")
# lemma.lemmatize("administer")

'abandonwar'

In [ ]:
#######################Stats memo##########################
No. of tokens: training set / test set

No operation: 62853 / 41462
    
spelling-correction: 
    
remove stopwords: 62721 / 41331
remove non-English: 20713 / 16111
remove non-Englsih/remove stopwords: 20587 / 15986
    
lemma/remove non-English: 21125 / 16518
stem/remove non-English: 10462 / 8352

    
lemma/remove non-English/remove stopwords: 21002 / 16396
stem/remove non-English/remove stopwords: 10359 / 8250

lemma/stem/remove non-English/remove stopwords: 10323 / 8236


    
